In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

# Load environmental variables (.env):
#   - GOOGLE_APPLICATION_CREDENTIALS
#   - GOOGLE_API_KEY
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Create a LLM
from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(model="gemini-pro")

In [4]:
from langchain_community.document_loaders import DirectoryLoader

# Load context files from directory:
loader = DirectoryLoader(
    path="../data",
    glob="**/*.context"
)
docs = loader.load()

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# Chunk and index loaded documents:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))

/home/ckhill/miniforge3/envs/deh/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")  # Default prompt to replace later.


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the RAG processing chain:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
rag_chain.invoke("What is hallucination in LLMs?")

'I am sorry, but the provided context does not contain information about hallucination in LLMs. Therefore, I am unable to answer your question. \n'